# EP1
## Classificação da clareza das respostas na plataforma eSIC

ACH2118 - Introdução ao Processamento de Língua Natural

Professor Ivandré Paraboni

Integrantes:
* Luiza Borghi de Mello - 11796037
* Raphael Nobuaki Iwamoto - 11882986


### Base

In [1]:
%pip install pandas
%pip install numpy
%pip install spacy
%pip install nltk
%pip install scikit-learn
%pip install openpyxl

!python -m spacy download pt_core_news_lg
!python3 -m spacy download pt_core_news_lg

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


You should consider upgrading via the 'c:\Users\Raphael\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
Python n�o encontrado; execute sem argumentos para instalar na Microsoft Store ou desabilite este atalho a partir de Configura��es > Gerenciar Aliases de Execu��o do Aplicativo.


In [2]:
import pandas as pd
import numpy as np
import spacy
import nltk
import time
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectPercentile, chi2, f_classif 

In [3]:
nlp = spacy.load('pt_core_news_lg')

def aplica_lematizacao(df: pd.DataFrame, coluna: str):
    df[coluna + '_lematizada'] = ''
    i = 0
    for text in df[coluna]:
        doc = nlp(text)
        lemma = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
        df[coluna + '_lematizada'].iloc[i] = " ".join(x for x in lemma)
        i = i + 1

In [4]:
nltk.download('rslp')

def aplica_stemming(df: pd.DataFrame, coluna: str):
    stemmer = nltk.stem.RSLPStemmer()
    df[coluna + '_stemming'] = df[coluna].apply(lambda x: " ".join(stemmer.stem(x) for x in x.split()))

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Raphael\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [5]:
nltk.download('stopwords')

def remove_stopwords(df: pd.DataFrame, coluna: str):
    stop = nltk.corpus.stopwords.words('portuguese')
    df[coluna + '_stopwords'] = df[coluna].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Raphael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Leitura e análise do dataset

In [6]:
treino_dataset = pd.read_excel('data/ep1_esic2023_clareza_TRAIN.xlsx')
treino_dataset

,resp_text,clarity
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234
...,...,...
5995,Trata-se de solicitação com base na Lei de Ac...,c1
5996,Trata-se de uma solicitação repetida. As info...,c5
5997,Unidade:,c5
5998,Vale dizer que a gestão dos Telefones de Uso ...,c234


In [7]:
treino_dataset.describe()

,resp_text,clarity
count,6000,6000
unique,5626,3
top,"Prezado(a) Senhor(a), Sua manifestação foi a...",c5
freq,41,2000


In [8]:
counts = treino_dataset['clarity'].value_counts()
counts

clarity
c5      2000
c234    2000
c1      2000
Name: count, dtype: int64

In [9]:
aplica_lematizacao(treino_dataset, 'resp_text')
aplica_stemming(treino_dataset, 'resp_text')
remove_stopwords(treino_dataset, 'resp_text')

treino_dataset

,resp_text,clarity,resp_text_lematizada,resp_text_stemming,resp_text_stopwords
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,Prezado Sr Jose Taunai em atenção a o seu pedi...,prez sr jos taun em atenç ao seu ped de acess ...,Prezado Sr Jose Taunai Em atenção pedido acess...
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,a pedido de o Pró-Reitor de Graduação informar...,"""a ped do pró-rei de graduação, inform que a u...","""A pedido Pró-Reitor Graduação, informamos UFS..."
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,Prezado o sr. o Agradecemos o contato e inform...,"""prez (a) sr. (a), agradec o contat e inform q...","""Prezado (a) Sr. (a), Agradecemos contato info..."
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,Prezado o sr. o Agradecemos o contato e inform...,"""prez (a) sr. (a), agradec o contat e inform q...","""Prezado (a) Sr. (a), Agradecemos contato info..."
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,Prezado Prof Gilberto Tadeu Reis de o Silva em...,"""prez prof. gilbert tad reil da silv em atend ...","""Prezado Prof. Gilberto Tadeu Reis Silva Em at..."
...,...,...,...,...,...
5995,Trata-se de solicitação com base na Lei de Ac...,c1,tratar se de solicitação com base em o Lei de ...,trata-s de solicit com bas na lei de acess à i...,Trata-se solicitação base Lei Acesso Informaçã...
5996,Trata-se de uma solicitação repetida. As info...,c5,tratar se de um solicitação repetir o informaç...,trata-s de uma solicit repetida. as inform req...,Trata-se solicitação repetida. As informações ...
5997,Unidade:,c5,Unidade,unidade:,Unidade:
5998,Vale dizer que a gestão dos Telefones de Uso ...,c234,Vale dizer que o gestão de o Telefones de Uso ...,val diz que a gest do telefon de uso públic é ...,Vale dizer gestão Telefones Uso Público feita ...


### Grid Search

#### 1. Logistic Regression | sem pré-processamento | Count Vectorizer

In [10]:
X = treino_dataset.resp_text
Y = treino_dataset.clarity

lowercase_list = [True, False]
analyzer_list = ['word', 'char']
ngram_range_list = [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
C_list = [0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0]
class_weight_list = ['balanced', None]
penalty_list = ['l1', 'l2']
solver_list = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

best_accuracy = 0
best_hyperparameters = {}

for lowercase, analyzer, ngram_range, C, class_weight, penalty, solver in itertools.product(lowercase_list, analyzer_list, ngram_range_list, C_list, class_weight_list, penalty_list, solver_list):
    vectorizer = CountVectorizer(analyzer=analyzer, lowercase=lowercase, ngram_range=ngram_range)
    X_count = vectorizer.fit_transform(X)

    clf = LogisticRegression(max_iter=9999, C=C, class_weight=class_weight, penalty=penalty, solver=solver, random_state=100)

    accuracy = 0
    try:
        accuracy = cross_val_score(clf, X_count, Y, scoring='accuracy', cv=10, error_score='raise').mean()

        print(f"Hyperparameters: lowercase={lowercase}, analyzer={analyzer}, ngram_range={ngram_range}, C={C}, class_weight={class_weight}, penalty={penalty}, solver={solver}")
        print(f"Mean Accuracy: {accuracy}\n")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = {
                'lowercase': lowercase,
                'analyzer': analyzer,
                'ngram_range': ngram_range,
                'C': C,
                'class_weight': class_weight,
                'penalty': penalty,
                'solver': solver
            }
    except:
        print('Combination skipped\n')

print("Best Hyperparameters:", best_hyperparameters)
print("Best Mean Accuracy:", best_accuracy)

X_count shape: (6000, 28710)
Combination skipped

X_count shape: (6000, 28710)
Hyperparameters: lowercase=True, analyzer=word, ngram_range=(1, 1), C=0.001, class_weight=balanced, penalty=l1, solver=liblinear
Mean Accuracy: 0.3321666666666666

X_count shape: (6000, 28710)
Combination skipped

X_count shape: (6000, 28710)
Combination skipped

X_count shape: (6000, 28710)
Combination skipped

X_count shape: (6000, 28710)
Hyperparameters: lowercase=True, analyzer=word, ngram_range=(1, 1), C=0.001, class_weight=balanced, penalty=l1, solver=saga
Mean Accuracy: 0.387

X_count shape: (6000, 28710)
Hyperparameters: lowercase=True, analyzer=word, ngram_range=(1, 1), C=0.001, class_weight=balanced, penalty=l2, solver=lbfgs
Mean Accuracy: 0.45

X_count shape: (6000, 28710)
Hyperparameters: lowercase=True, analyzer=word, ngram_range=(1, 1), C=0.001, class_weight=balanced, penalty=l2, solver=liblinear
Mean Accuracy: 0.4521666666666667

X_count shape: (6000, 28710)
Hyperparameters: lowercase=True, an

#### 2. Logistic Regression | Lematização | TF-IDF Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

lowercase_list = [True, False]
analyzer_list = ['word', 'char']
ngram_range_list = [(2, 2), (3, 3), (4, 4), (5, 5)]
C_list = np.geomspace(3.0, 5.0, num=20)
class_weight_list = ['balanced', None]
penalty_list = ['l1', 'l2']
solver_list = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

best_accuracy = 0
best_hyperparameters = {}

for lowercase, analyzer, ngram_range, C, class_weight, penalty, solver in itertools.product(lowercase_list, analyzer_list, ngram_range_list, C_list, class_weight_list, penalty_list, solver_list):
    vectorizer = TfidfVectorizer(analyzer=analyzer, lowercase=lowercase, ngram_range=ngram_range)
    X_tfidf = vectorizer.fit_transform(X)

    clf = LogisticRegression(max_iter=9999, C=C, class_weight=class_weight, penalty=penalty, solver=solver, random_state=100)

    accuracy = 0
    try:
        accuracy = cross_val_score(clf, X_tfidf, Y, scoring='accuracy', cv=10, error_score='raise').mean()

        print(f"Hyperparameters: lowercase={lowercase}, analyzer={analyzer}, ngram_range={ngram_range}, C={C}, class_weight={class_weight}, penalty={penalty}, solver={solver}")
        print(f"Mean Accuracy: {accuracy}\n")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = {
                'lowercase': lowercase,
                'analyzer': analyzer,
                'ngram_range': ngram_range,
                'C': C,
                'class_weight': class_weight,
                'penalty': penalty,
                'solver': solver
            }
    except:
        print('Combination skipped\n')

print("Best Hyperparameters:", best_hyperparameters)
print("Best Mean Accuracy:", best_accuracy)

#### 3. Naive Bayes | sem pré-processamento | Count Vectorizer

In [ ]:
X = treino_dataset.resp_text
Y = treino_dataset.clarity

analyzer_list = ['word', 'char']
ngram_range_list = [(1, 1), (2, 2), (3, 3), (5, 5)]
alpha_list = np.geomspace(1e-3, 1, num=50)

best_accuracy = 0
best_hyperparameters = {}

for analyzer, ngram_range, alpha in itertools.product(analyzer_list, ngram_range_list, alpha_list):
    vectorizer = CountVectorizer(ngram_range=ngram_range, analyzer=analyzer)
    X_count = vectorizer.fit_transform(X)

    clf = MultinomialNB(alpha=alpha)

    accuracy = cross_val_score(clf, X_count, Y, scoring='accuracy', cv=10).mean()

    print(f"Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}, alpha={alpha}")
    print(f"Mean Accuracy: {accuracy}\n")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'ngram_range': ngram_range,
            'analyzer': analyzer,
            'alpha': alpha
        }

print("Best Hyperparameters:", best_hyperparameters)
print("Best Mean Accuracy:", best_accuracy)

#### 4. Naive Bayes | Lematização | TF-IDF Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

analyzer_list = ['word', 'char']
ngram_range_list = [(1, 1), (2, 2), (3, 3), (5, 5)]
alpha_list = np.geomspace(1e-3, 1000, num=100)

best_accuracy = 0
best_hyperparameters = {}

for analyzer, ngram_range, alpha in itertools.product(analyzer_list, ngram_range_list, alpha_list):
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, analyzer=analyzer)
    X_tfidf = vectorizer.fit_transform(X)

    clf = MultinomialNB(alpha=alpha)

    accuracy = cross_val_score(clf, X_tfidf, Y, scoring='accuracy', cv=10).mean()

    print(f"Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}, alpha={alpha}")
    print(f"Mean Accuracy: {accuracy}\n")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'ngram_range': ngram_range,
            'analyzer': analyzer,
            'alpha': alpha
        }

print("Best Hyperparameters:", best_hyperparameters)
print("Best Mean Accuracy:", best_accuracy)

#### 5. Random Forest | Lematização | Count Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

analyzer = 'word'
ngram_range = (1, 1)

vectorizer = CountVectorizer(ngram_range=ngram_range, analyzer=analyzer)
X_count = vectorizer.fit_transform(X)

clf = RandomForestClassifier(random_state=100)

accuracy = cross_val_score(clf, X, Y, scoring='accuracy', cv=10, n_jobs=3).mean()

print(f"Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}")
print(f"Mean Accuracy: {accuracy}\n")

#### 6. Random Forest | Lematização | TF-IDF Vectorizer

In [ ]:
X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

analyzer_list = ['word', 'char']
ngram_range_list = [(1, 1), (2, 2)]

best_accuracy = 0
best_hyperparameters = {}

for analyzer, ngram_range in itertools.product(analyzer_list, ngram_range_list):
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, analyzer=analyzer)
    X_tfidf = vectorizer.fit_transform(X)

    clf = RandomForestClassifier(random_state=100)

    accuracy = cross_val_score(clf, X, Y, scoring='accuracy', cv=10).mean()

    print(f"Hyperparameters: ngram_range={ngram_range}, analyzer={analyzer}")
    print(f"Mean Accuracy: {accuracy}\n")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'ngram_range': ngram_range,
            'analyzer': analyzer,
        }

print("Best Hyperparameters:", best_hyperparameters)
print("Best Mean Accuracy:", best_accuracy)

#### 7. SVC | Lematização | Count Vectorizer

#### 8. SVC | Lematização | TF-IDF Vectorizer

#### 9. SGD Classifier | Lematização | TF-IDF Vectorizer

In [10]:
X = treino_dataset.resp_text
Y = treino_dataset.clarity

alpha_list = np.geomspace(6.632895500464648e-05, 0.00014590812272681345, num=30)
max_iter_list = [100, 1000]

best_accuracy = 0
best_hyperparameters = {}

for alpha, max_iter in itertools.product(alpha_list, max_iter_list):
    vectorizer = TfidfVectorizer(max_features=None, ngram_range=(3, 3), analyzer='char', lowercase=True)
    X_tfidf = vectorizer.fit_transform(X)

    clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=alpha, random_state=100, max_iter=max_iter, tol=None)
    try:
        accuracy = cross_val_score(clf, X_tfidf, Y, scoring='accuracy', cv=10, n_jobs=4).mean()

        print(f"Hyperparameters: alpha={alpha}, max_iter={max_iter}")
        print(f"Mean Accuracy: {accuracy}\n")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = {
                'alpha': alpha,
                'max_iter': max_iter
            }
    except:
        print('Combination skipped\n')

print("Best Hyperparameters:", best_hyperparameters)
print("Best Mean Accuracy:", best_accuracy)

Combination skipped



#### 10. SGD Classifier | Lematização | TF-IDF Vectorizer | SelectPercentile

In [10]:
X = treino_dataset.resp_text
Y = treino_dataset.clarity

alpha_list = np.geomspace(6.632895500464648e-05, 0.00014590812272681345, num=30)
max_iter_list = [100, 1000]
percentile_list = [50, 75, 100]
score_func_list = [chi2, f_classif]

best_accuracy = 0
best_hyperparameters = {}

for alpha, max_iter, percentile, score_func in itertools.product(alpha_list, max_iter_list, percentile_list, score_func_list):
  vectorizer = TfidfVectorizer(max_features=None, ngram_range=(3, 3), analyzer='char', lowercase=True)
  X_tfidf = vectorizer.fit_transform(X)
  X_BestFeatures = SelectPercentile(score_func=score_func , percentile=percentile).fit_transform(X_tfidf, Y)

  clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=alpha, random_state=100, max_iter=max_iter, tol=None)
  try:
    accuracy = cross_val_score(clf, X_BestFeatures, Y, scoring='accuracy', cv=10, n_jobs=4).mean()

    print(f"Hyperparameters: alpha={alpha}, max_iter={max_iter}, percentile={percentile}, score_func={score_func}")
    print(f"Mean Accuracy: {accuracy}\n")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {
            'alpha': alpha,
            'max_iter': max_iter,
            'percentile': percentile,
            'score_func': score_func
        }
  except:
      print('Combination skipped\n')  

print("Best Hyperparameters:", best_hyperparameters)
print("Best Mean Accuracy:", best_accuracy)

Hyperparameters: alpha=6.632895500464648e-05, max_iter=100, percentile=50, score_func=<function chi2 at 0x0000020DA3774700>
Mean Accuracy: 0.49883333333333324

Hyperparameters: alpha=6.632895500464648e-05, max_iter=100, percentile=50, score_func=<function f_classif at 0x0000020DA3774550>
Mean Accuracy: 0.4998333333333333

Hyperparameters: alpha=6.632895500464648e-05, max_iter=100, percentile=75, score_func=<function chi2 at 0x0000020DA3774700>
Mean Accuracy: 0.49066666666666664

Hyperparameters: alpha=6.632895500464648e-05, max_iter=100, percentile=75, score_func=<function f_classif at 0x0000020DA3774550>
Mean Accuracy: 0.5006666666666667

Hyperparameters: alpha=6.632895500464648e-05, max_iter=100, percentile=100, score_func=<function chi2 at 0x0000020DA3774700>
Mean Accuracy: 0.472

Hyperparameters: alpha=6.632895500464648e-05, max_iter=100, percentile=100, score_func=<function f_classif at 0x0000020DA3774550>
Mean Accuracy: 0.472

Hyperparameters: alpha=6.632895500464648e-05, max_ite

#### 10. test

In [23]:
from sklearn.feature_selection import SelectKBest, chi2

X = treino_dataset.resp_text_lematizada
Y = treino_dataset.clarity

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)
print(f"X_Tfidf shape: {X_tfidf.shape}")

X_Best = SelectKBest(score_func=chi2, k=5000).fit_transform(X_tfidf, Y)
print(f"X_Best shape: {X_Best.shape}")

clf = LogisticRegression(max_iter=9999)

accuracy = cross_val_score(clf, X_tfidf, Y, scoring='accuracy', cv=10, error_score='raise').mean()
print(f"TFIDF: {accuracy}")
accuracy = cross_val_score(clf, X_Best, Y, scoring='accuracy', cv=10, error_score='raise').mean()
print(f"X_Best: {accuracy}")



X_Tfidf shape: (6000, 24782)
X_Best shape: (6000, 5000)
TFIDF: 0.4623333333333333
X_Best: 0.505
